# Proyecto 1 — Estación de llenado y taponado



Importación de las librerias

In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

Definir las rutas de los archivos

In [ ]:
data_path = Path("data")

telemetria_file = data_path / "telemetria.csv"
eventos_file = data_path / "eventos.csv"
botellas_file = data_path / "botellas.csv"

# FASE 1 --->Ingesta y validación (Pandas)


PASO 1: Carga y tipado de datos

    1.1 Definir tipos de datos explícitos para cada CSV utilizando un diccionario 

In [ ]:

dtype_tel = {
    'temp_prod': 'float32',
    'vel_cinta': 'float32',
    'caudal': 'float32',
    'energia_kwh': 'float64'
}

dtype_evt = {
    'tipo': 'category',
    'id_botella': 'Int64'
}

dtype_pz = {
    'id_botella': 'int64',
    'peso_neto': 'float32',
    'formato': 'category'
}


    1.2 Cargar los csv en los dataFrames parseando el tiempo a datetime64

In [ ]:
# Cargar telemetría con parseo de fecha
df_tel = pd.read_csv(
    telemetria_file,
    dtype=dtype_tel,
    parse_dates=['ts'],
    date_format='ISO8601'
)
# Convertir ts a UTC y establecer como índice
df_tel['ts'] = pd.to_datetime(df_tel['ts'], utc=True) # Convierte a datetime con zona horaria UTC
df_tel = df_tel.set_index('ts').sort_index() # Establece la columna de tiempo como índice del DataFrame

# Cargar eventos
df_evt = pd.read_csv(
    eventos_file,
    dtype=dtype_evt,
    parse_dates=['ts_ini', 'ts_fin'],
    date_format='ISO8601'
) # Lee el CSV y convierte automáticamente las columnas de fecha
df_evt['ts_ini'] = pd.to_datetime(df_evt['ts_ini'], utc=True) #Convierte a datetime con zona horaria UTC
df_evt['ts_fin'] = pd.to_datetime(df_evt['ts_fin'], utc=True) #Convierte a datetime con zona horaria UTC
df_evt = df_evt.sort_values('ts_ini').reset_index(drop=True)

# Cargar botellas
df_pz = pd.read_csv(
    botellas_file,
    dtype=dtype_pz,
    parse_dates=['ts_ciclo'],
    date_format='ISO8601'
)
df_pz['ts_ciclo'] = pd.to_datetime(df_pz['ts_ciclo'], utc=True) #Convierte a datetime con zona horaria UTC
df_pz = df_pz.sort_values('ts_ciclo').reset_index(drop=True) #


PASO 2. Orden y duplicados

    2.1 Telemetría

In [ ]:
print("Telemetria:")
# Eliminar duplicados exactos en telemetría
duplicados_antes_tel = df_tel.duplicated().sum()
df_tel = df_tel[~df_tel.duplicated(keep='first')]

#df_evt = df_evt[~df_evt.duplicated(keep='first')]

# Verificar monotonía del índice
es_monotono_tel = df_tel.index.is_monotonic_increasing #Comprobacion de la monotonia: Los indices temporales avanzan correctamente de menor a mayor

print(f"Duplicados eliminados: {duplicados_antes_tel}")
print(f"Índice monótono: {es_monotono_tel}")

# Verificar si hay duplicados en el índice temporal
duplicados_index = df_tel.index.duplicated().sum()
if duplicados_index > 0:
    print(f"Hay {duplicados_index} timestamps duplicados en el índice")
    df_tel = df_tel[~df_tel.index.duplicated(keep='first')]
else:
    print("No hay timestamps duplicados")


    2.2 Eventos

In [ ]:
print("Eventos:")
# Eliminar duplicados exactos en telemetría
dup_evt = df_evt.duplicated().sum()
df_evt = df_evt[~df_evt.duplicated(keep='first')]

# Ordenar por tiempo de inicio (y fin como desempate)
df_evt = df_evt.sort_values(['ts_ini', 'ts_fin']).reset_index(drop=True)

# Chequeos
es_monotono_evt = df_evt['ts_ini'].is_monotonic_increasing
neg_dur = (df_evt['ts_fin'] < df_evt['ts_ini']).sum()
dup_ts_ini = df_evt['ts_ini'].duplicated().sum()

print(f"Duplicados eliminados: {dup_evt}")
print(f"Orden por ts_ini monótono: {es_monotono_evt}")
print(f"Eventos con ts_fin < ts_ini: {neg_dur}")
print(f"Timestamps ts_ini duplicados: {dup_ts_ini}")


    2.3 Botellas

In [ ]:
print("Botellas:")
dup_pz = df_pz.duplicated().sum()
df_pz = df_pz[~df_pz.duplicated(keep='first')]

# Ordenar por ts_ciclo
df_pz = df_pz.sort_values('ts_ciclo').reset_index(drop=True)

# Chequeos
es_monotono_pz = df_pz['ts_ciclo'].is_monotonic_increasing
dup_ts_ciclo = df_pz['ts_ciclo'].duplicated().sum()
dup_id_botella = df_pz['id_botella'].duplicated().sum()

print(f"Duplicados eliminados: {dup_pz}")
print(f"Orden por ts_ciclo monótono: {es_monotono_pz}")
print(f"Timestamps ts_ciclo duplicados: {dup_ts_ciclo}")
print(f"id_botella duplicados: {dup_id_botella}")

PASO 3: Validaciones de rango

    3.1 Marcar valores fuera de rango (sin eliminar)

In [ ]:
RANGO_TEMP = (18.0,35.0)
RANGO_VEL = (0.0,0.5)
RANGO_CAUDAL = (0.0,12.0)

df_tel['fuera_RANGO_TEMP'] = (df_tel['temp_prod'] < RANGO_TEMP[0]) | (df_tel['temp_prod'] > RANGO_TEMP[1])
df_tel['fuera_RANGO_VEL'] = (df_tel['vel_cinta'] < RANGO_VEL[0]) | (df_tel['vel_cinta'] > RANGO_VEL[1])
df_tel['fuera_RANGO_CAUDAL'] = (df_tel['caudal'] < RANGO_CAUDAL[0]) | (df_tel['caudal'] > RANGO_CAUDAL[1])

n_temp_fuera = df_tel['fuera_RANGO_TEMP'].sum()
n_vel_fuera = df_tel['fuera_RANGO_VEL'].sum()
n_caudal_fuera = df_tel['fuera_RANGO_CAUDAL'].sum()

print("="*60)
print("VALIDACIÓN DE RANGOS")
print("="*60)
print(f"temp_prod fuera de [{RANGO_TEMP[0]}, {RANGO_TEMP[1]}] °C: {n_temp_fuera} ({n_temp_fuera/len(df_tel)*100:.2f}%)")
print(f"vel_cinta fuera de [{RANGO_VEL[0]}, {RANGO_VEL[1]}] m/s: {n_vel_fuera} ({n_vel_fuera/len(df_tel)*100:.2f}%)")
print(f"caudal fuera de [{RANGO_CAUDAL[0]}, {RANGO_CAUDAL[1]}] ml/s: {n_caudal_fuera} ({n_caudal_fuera/len(df_tel)*100:.2f}%)")

# Mostrar estadísticas descriptivas
print("\nEstadísticas descriptivas:")
print(df_tel[['temp_prod', 'vel_cinta', 'caudal', 'energia_kwh']].describe())

    3.2 Validar que energia_kwh no decrece (salvo cuantización)

In [ ]:
# Calcular diferencias entre valores consecutivos de energía
df_tel['delta_energia'] = df_tel['energia_kwh'].diff()

# Contar cuántas veces la energía DECRECE (delta < 0)
# Nota: diff() genera NaN en la primera fila, lo ignoramos
decrementos = (df_tel['delta_energia'] < 0).sum() 
total_cambios = df_tel['delta_energia'].notna().sum() #Va a contar cuantas veces decrece

print("="*60)
print("VALIDACIÓN DE ENERGÍA NO DECRECIENTE")
print("="*60)
print(f"Total de cambios: {total_cambios:,}")
print(f"Decrementos detectados: {decrementos} ({decrementos/total_cambios*100:.3f}%)")

# Mostrar algunos ejemplos de decrementos (si existen)
if decrementos > 0:
    print("\n Ejemplos de energía que decrece:")
    ejemplos_decremento = df_tel[df_tel['delta_energia'] < 0][['energia_kwh', 'delta_energia']].head(10)
    print(ejemplos_decremento)
    
    # Estadísticas de los decrementos
    print("\n Estadísticas de los decrementos:")
    print(df_tel[df_tel['delta_energia'] < 0]['delta_energia'].describe())
else:
    print("\n No se detectaron decrementos en energia_kwh")

PASO 4: Monotonicidad de energía (corrección de decrementos)

In [ ]:
# PASO 4: Corregir decrementos de energía (si los hubiera)
# Guardamos la columna original para comparación
df_tel['energia_kwh_original'] = df_tel['energia_kwh'].copy()

# Identificar decrementos
decrementos_mask = df_tel['delta_energia'] < 0
n_correcciones = decrementos_mask.sum()

if n_correcciones > 0:
    print(f"Se encontraron {n_correcciones} decrementos. Corrigiendo...")
    
    # Hacer clip de deltas negativos a 0
    df_tel['delta_energia_corregida'] = df_tel['delta_energia'].clip(lower=0)
    
    # Reconstruir energía acumulada desde el primer valor
    energia_inicial = df_tel['energia_kwh'].iloc[0]
    df_tel['energia_kwh'] = energia_inicial + df_tel['delta_energia_corregida'].fillna(0).cumsum()
    
    # Recalcular delta_energia con valores corregidos
    df_tel['delta_energia'] = df_tel['energia_kwh'].diff()
    
    print(f"{n_correcciones} correcciones aplicadas")
else:
    print("No se requieren correcciones en energia_kwh")
    print("La señal es naturalmente monótona creciente")

# Verificación final
decrementos_final = (df_tel['delta_energia'] < 0).sum()
print(f"\nVerificación final: {decrementos_final} decrementos después de corrección")

PASO 5: Frecuencia y huecos temporales

    5.1 Confirmar frecuencia nominal de 1 Hz

In [ ]:
# 5.1 Analizar la frecuencia de muestreo
print("="*60)
print("ANÁLISIS DE FRECUENCIA DE MUESTREO")
print("="*60)

# Calcular diferencias de tiempo entre muestras consecutivas
time_diffs = df_tel.index.to_series().diff()

# Contar muestras con intervalo de 1 segundo
intervalo_1s = time_diffs == pd.Timedelta(seconds=1)
n_1s = intervalo_1s.sum()
total = len(time_diffs) - 1  # -1 porque el primer valor es NaN

print(f"\nMuestras con intervalo de 1s: {n_1s}/{total}")

# Identificar huecos (intervalos > 1s)
huecos = time_diffs[time_diffs > pd.Timedelta(seconds=1)]
n_huecos = len(huecos)

print(f"\nHuecos detectados (intervalos > 1s): {n_huecos}")

if n_huecos > 0:
    print(f"\nEstadísticas de los huecos:")
    print(huecos.describe())
    
    # Clasificar huecos
    huecos_pequenos = huecos[huecos <= pd.Timedelta(seconds=10)]
    huecos_grandes = huecos[huecos > pd.Timedelta(seconds=10)]
    
    print(f"\nHuecos pequeños (≤10s): {len(huecos_pequenos)}")
    print(f"Huecos grandes (>10s): {len(huecos_grandes)}")
else:
    print(f"El numero de huecos es: {n_huecos}")
    print("La frecuencia nominal es de 1 Hz. Todos los saltos son de un segundo")
    print("No es necesario interpolar ni marcar segmentos invalidos")


    5.2 Reindexar a rejilla de 1 segundo

In [ ]:
# 5.2 Reindexar a rejilla regular de 1 segundo
print("\n" + "="*60)
print("REINDEXACIÓN A REJILLA DE 1 SEGUNDO")
print("="*60)

# Crear rejilla temporal de 1s desde el primer al último timestamp
ts_inicio = df_tel.index.min()
ts_fin = df_tel.index.max()
rejilla_1s = pd.date_range(start=ts_inicio, end=ts_fin, freq='1s')

print(f"\nRango temporal:")
print(f"   Inicio: {ts_inicio}")
print(f"   Fin: {ts_fin}")
print(f"   Duración: {ts_fin - ts_inicio}")

print(f"\nTamaño de los datos:")
print(f"   Muestras originales: {len(df_tel):,}")
print(f"   Rejilla de 1s: {len(rejilla_1s):,}")
print(f"   Diferencia (huecos): {len(rejilla_1s) - len(df_tel):,}")

# Reindexar el DataFrame a la rejilla de 1s
df_tel = df_tel.reindex(rejilla_1s)

print(f"\nDataFrame reindexado")

5.3 Rellenar huecos pequeños (≤10s) con interpolación

In [ ]:
# 5.3 Rellenar huecos ≤ 10s
print("\n" + "="*60)
print("RELLENADO DE HUECOS PEQUEÑOS (≤10s)")
print("="*60)

# Identificar bloques de NaN consecutivos
df_tel['es_nan'] = df_tel['temp_prod'].isna()
df_tel['bloque_nan'] = (df_tel['es_nan'] != df_tel['es_nan'].shift()).cumsum()

# Calcular tamaño de cada bloque de NaN
tamano_bloques = df_tel[df_tel['es_nan']].groupby('bloque_nan').size()

# Clasificar bloques
bloques_pequenos = tamano_bloques[tamano_bloques <= 10]
bloques_grandes = tamano_bloques[tamano_bloques > 10]

print(f"\nBloques de NaN detectados:")
print(f"   Total de bloques: {len(tamano_bloques)}")
print(f"   Bloques ≤10s: {len(bloques_pequenos)} (se interpolarán)")
print(f"   Bloques >10s: {len(bloques_grandes)} (se marcarán como inválidos)")

# Crear máscara para huecos pequeños (≤10s)
mask_huecos_pequenos = df_tel['bloque_nan'].isin(bloques_pequenos.index) & df_tel['es_nan']

# Interpolación lineal para temp_prod y caudal en huecos pequeños
print(f"\nInterpolando temp_prod y caudal...")
df_tel.loc[mask_huecos_pequenos, 'temp_prod'] = df_tel['temp_prod'].interpolate(method='linear', limit=10)
df_tel.loc[mask_huecos_pequenos, 'caudal'] = df_tel['caudal'].interpolate(method='linear', limit=10)

# Forward-fill para vel_cinta (propagar último valor válido)
print(f"Forward-fill en vel_cinta...")
df_tel.loc[mask_huecos_pequenos, 'vel_cinta'] = df_tel['vel_cinta'].ffill(limit=10)


    5.4 Marcar huecos grandes (>10s) como inválidos

In [ ]:
# 5.4 Marcar huecos grandes como inválidos
print("\n" + "="*60)
print("MARCADO DE HUECOS GRANDES (>10s)")
print("="*60)

# Crear columna para marcar segmentos inválidos
mask_huecos_grandes = df_tel['bloque_nan'].isin(bloques_grandes.index) & df_tel['es_nan']
df_tel['segmento_invalido'] = mask_huecos_grandes

# Contar segundos marcados como inválidos
n_invalidos = df_tel['segmento_invalido'].sum()
total_segundos = len(df_tel)

print(f"\nSegmentos marcados como inválidos:")
print(f"   Total de segundos inválidos: {n_invalidos:,}")
print(f"   Porcentaje: {n_invalidos/total_segundos*100:.2f}%")

if len(bloques_grandes) > 0:
    print(f"\nDetalle de huecos grandes:")
    for i, (bloque_id, tamano) in enumerate(bloques_grandes.items(), 1):
        inicio_hueco = df_tel[df_tel['bloque_nan'] == bloque_id].index.min()
        fin_hueco = df_tel[df_tel['bloque_nan'] == bloque_id].index.max()
        print(f"   Hueco {i}: {tamano}s desde {inicio_hueco} hasta {fin_hueco}")
        if i >= 5:
            print(f"   ... y {len(bloques_grandes)-5} huecos más")
            break

# Limpiar columnas auxiliares
df_tel = df_tel.drop(columns=['es_nan', 'bloque_nan'])

print(f"\n✅ Proceso de huecos completado")

PASO 6: Detección de atípicos

    6.1 Detección por z-score (umbral ±3)

In [ ]:
# 6.1 Detección de atípicos por z-score
print("="*60)
print("DETECCIÓN DE ATÍPICOS POR Z-SCORE")
print("="*60)

# Umbral estándar: valores con |z-score| > 3 son atípicos
UMBRAL_Z = 3

# Calcular z-score para cada variable
# z-score = (valor - media) / desviación estándar
df_tel['z_temp'] = (df_tel['temp_prod'] - df_tel['temp_prod'].mean()) / df_tel['temp_prod'].std()
df_tel['z_vel'] = (df_tel['vel_cinta'] - df_tel['vel_cinta'].mean()) / df_tel['vel_cinta'].std()
df_tel['z_caudal'] = (df_tel['caudal'] - df_tel['caudal'].mean()) / df_tel['caudal'].std()

# Marcar atípicos (|z| > 3)
df_tel['atipico_z_temp'] = df_tel['z_temp'].abs() > UMBRAL_Z
df_tel['atipico_z_vel'] = df_tel['z_vel'].abs() > UMBRAL_Z
df_tel['atipico_z_caudal'] = df_tel['z_caudal'].abs() > UMBRAL_Z

# Contar atípicos detectados
n_atip_temp = df_tel['atipico_z_temp'].sum()
n_atip_vel = df_tel['atipico_z_vel'].sum()
n_atip_caudal = df_tel['atipico_z_caudal'].sum()

print(f"\nAtípicos detectados (|z-score| > {UMBRAL_Z}):")
print(f"   temp_prod: {n_atip_temp} ({n_atip_temp/len(df_tel)*100:.3f}%)")
print(f"   vel_cinta: {n_atip_vel} ({n_atip_vel/len(df_tel)*100:.3f}%)")
print(f"   caudal: {n_atip_caudal} ({n_atip_caudal/len(df_tel)*100:.3f}%)")

# Mostrar ejemplos si existen
if n_atip_temp > 0:
    print("\nEjemplos de atípicos en temp_prod:")
    print(df_tel[df_tel['atipico_z_temp']][['temp_prod', 'z_temp']].head())

    6.2 Detección por IQR (rango intercuartílico)

In [ ]:
# 6.2 Detección de atípicos por IQR
print("\n" + "="*60)
print("DETECCIÓN DE ATÍPICOS POR IQR")
print("="*60)

# Calcular cuartiles y rango intercuartílico (IQR)
# IQR = Q3 - Q1
# Límites: [Q1 - 1.5*IQR, Q3 + 1.5*IQR]

for var in ['temp_prod', 'vel_cinta', 'caudal']:
    Q1 = df_tel[var].quantile(0.25)
    Q3 = df_tel[var].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    # Marcar atípicos
    col_name = f'atipico_iqr_{var.split("_")[0]}'  # atipico_iqr_temp, atipico_iqr_vel, atipico_iqr_caudal
    df_tel[col_name] = (df_tel[var] < limite_inferior) | (df_tel[var] > limite_superior)
    
    n_atipicos = df_tel[col_name].sum()
    
    print(f"\n{var}:")
    print(f"   Q1: {Q1:.3f}")
    print(f"   Q3: {Q3:.3f}")
    print(f"   IQR: {IQR:.3f}")
    print(f"   Límites: [{limite_inferior:.3f}, {limite_superior:.3f}]")
    print(f"   Atípicos: {n_atipicos} ({n_atipicos/len(df_tel)*100:.3f}%)")

    6.3 Consolidar marcas de atípicos

In [ ]:
# 6.3 Consolidar detección de atípicos
print("\n" + "="*60)
print("CONSOLIDACIÓN DE ATÍPICOS")
print("="*60)

# Crear columna que marca si hay algún atípico (OR lógico)
# Un registro es atípico si al menos una variable lo es (por cualquier método)
df_tel['es_atipico'] = (
    df_tel['atipico_z_temp'] | df_tel['atipico_z_vel'] | df_tel['atipico_z_caudal'] |
    df_tel['atipico_iqr_temp'] | df_tel['atipico_iqr_vel'] | df_tel['atipico_iqr_caudal']
)

total_atipicos = df_tel['es_atipico'].sum()
porcentaje = total_atipicos / len(df_tel) * 100

print(f"\nRegistros con al menos un valor atípico:")
print(f"   Total: {total_atipicos:,}")
print(f"   Porcentaje: {porcentaje:.2f}%")

# Resumen por método
print(f"\nComparación de métodos:")
print(f"   Z-score: {(df_tel['atipico_z_temp'] | df_tel['atipico_z_vel'] | df_tel['atipico_z_caudal']).sum():,}")
print(f"   IQR: {(df_tel['atipico_iqr_temp'] | df_tel['atipico_iqr_vel'] | df_tel['atipico_iqr_caudal']).sum():,}")

PASO 7: Etiqueta RUN/STOP por segundo

    7.1 Construir máscara de paradas desde eventos

In [ ]:
# 7.1 Construir máscara STOP_evt desde eventos.csv
print("="*60)
print("CONSTRUCCIÓN DE MÁSCARA RUN/STOP")
print("="*60)

# Filtrar eventos que implican parada
eventos_parada = df_evt[df_evt['tipo'].isin(['micro_parada', 'cambio_formato', 'limpieza'])].copy()

print(f"\nEventos de parada encontrados:")
print(f"   Total: {len(eventos_parada)}")
print(f"   micro_parada: {(eventos_parada['tipo'] == 'micro_parada').sum()}")
print(f"   cambio_formato: {(eventos_parada['tipo'] == 'cambio_formato').sum()}")
print(f"   limpieza: {(eventos_parada['tipo'] == 'limpieza').sum()}")

# Inicializar columna STOP_evt en False (por defecto está en marcha)
df_tel['STOP_evt'] = False

# Marcar como True los segundos que caen en intervalos [ts_ini, ts_fin)
for idx, evento in eventos_parada.iterrows():
    mascara_tiempo = (df_tel.index >= evento['ts_ini']) & (df_tel.index < evento['ts_fin'])
    df_tel.loc[mascara_tiempo, 'STOP_evt'] = True

n_stop_evt = df_tel['STOP_evt'].sum()
print(f"\nSegundos marcados como STOP por eventos: {n_stop_evt:,} ({n_stop_evt/len(df_tel)*100:.2f}%)")

7.2 Definir RUN basado en velocidad de cinta

In [ ]:
# 7.2 Definir RUN_vel basado en velocidad de cinta
print("\n" + "="*60)
print("DEFINICIÓN DE RUN_vel")
print("="*60)

# Umbral de velocidad para considerar que la máquina está en marcha
UMBRAL_VEL_RUN = 0.05  # m/s

# RUN_vel = True si vel_cinta >= 0.05 m/s
df_tel['RUN_vel'] = df_tel['vel_cinta'] >= UMBRAL_VEL_RUN

n_run_vel = df_tel['RUN_vel'].sum()
print(f"\nUmbral de velocidad: {UMBRAL_VEL_RUN} m/s")
print(f"Segundos con RUN_vel=True: {n_run_vel:,}")
print(f"Segundos con RUN_vel=False: {len(df_tel)-n_run_vel:,}")

7.3 Combinar en estado final (RUN/STOP)

In [ ]:
# 7.3 Definir estado final: RUN si RUN_vel=True Y STOP_evt=False
print("\n" + "="*60)
print("COMBINACIÓN DE CONDICIONES")
print("="*60)

# estado = RUN si (RUN_vel AND NOT STOP_evt), STOP en otro caso
df_tel['estado'] = 'STOP'
df_tel.loc[df_tel['RUN_vel'] & ~df_tel['STOP_evt'], 'estado'] = 'RUN'

# Convertir a tipo category para ahorrar memoria
df_tel['estado'] = df_tel['estado'].astype('category')

# Contar estados
n_run = (df_tel['estado'] == 'RUN').sum()
n_stop = (df_tel['estado'] == 'STOP').sum()

print(f"\nDistribución de estados:")
print(f"   RUN: {n_run:,}")
print(f"   STOP: {n_stop:,}")

# Análisis de transiciones
df_tel['cambio_estado'] = df_tel['estado'] != df_tel['estado'].shift()
n_transiciones = df_tel['cambio_estado'].sum() - 1  # -1 para excluir el primer valor

print(f"\nTransiciones de estado detectadas: {n_transiciones}")

PASO 8: Agregación a 1 minuto (diagnóstico temprano)

In [ ]:
print("="*60)
print("AGREGACIÓN A 1 MINUTO")
print("="*60)

# Crear agregaciones por minuto
df_1min = df_tel.resample('1min').agg({
    'temp_prod': ['mean', lambda x: x.quantile(0.95)],
    'caudal': 'mean',
    'vel_cinta': 'mean',
    'energia_kwh': 'last',  # Último valor del minuto (acumulado)
    'estado': lambda x: (x == 'STOP').sum()  # Contar segundos en STOP
}).round(3)

# Aplanar nombres de columnas
df_1min.columns = ['temp_mean', 'temp_p95', 'caudal_mean', 'vel_cinta_mean', 'energia_kwh', 'segundos_stop']

# Calcular métricas derivadas
df_1min['pct_stop'] = (df_1min['segundos_stop'] / 60 * 100).round(2)
df_1min['segundos_run'] = 60 - df_1min['segundos_stop']
df_1min['pct_run'] = (df_1min['segundos_run'] / 60 * 100).round(2)

# Calcular delta de energía por minuto
df_1min['delta_energia_min'] = df_1min['energia_kwh'].diff()

# Información del resultado
print(f"\nDataFrame agregado:")
print(f"   Registros originales (1s): {len(df_tel):,}")
print(f"   Registros agregados (1min): {len(df_1min):,}")
print(f"   Rango temporal: {df_1min.index.min()} a {df_1min.index.max()}")

print(f"\nColumnas creadas:")
for col in df_1min.columns:
    print(f"   - {col}")

print(f"\nEstadísticas de disponibilidad:")
print(f"   Media % RUN por minuto: {df_1min['pct_run'].mean():.2f}%")
print(f"   Media % STOP por minuto: {df_1min['pct_stop'].mean():.2f}%")
print(f"   Minutos con 100% RUN: {(df_1min['pct_run'] == 100).sum()} ({(df_1min['pct_run'] == 100).sum()/len(df_1min)*100:.2f}%)")
print(f"   Minutos con 100% STOP: {(df_1min['pct_stop'] == 100).sum()} ({(df_1min['pct_stop'] == 100).sum()/len(df_1min)*100:.2f}%)")

print(f"\nPrimeros registros:")
print(df_1min.head())

# FASE 2: Ingeniería de variables y KPIs (NumPy + Pandas)

PASO 1: Cálculo de potencia instantánea desde energía acumulada

    Fórmula: P_kW = ΔE / Δt (donde Δt está en horas)

### Fórmulas implementadas

**Cálculo de potencia instantánea:**

$$\Delta E_i = \max\{E_i - E_{i-1}, 0\}$$

$$\Delta t_i = \frac{t_i - t_{i-1}}{3600} \text{ (horas)}$$

$$P_{\text{kW},i} = \frac{\Delta E_i}{\Delta t_i}$$

$$P_{\text{W},i} = 1000 \cdot P_{\text{kW},i}$$

**Suavizado opcional (media móvil):**
- Ventana: 5 segundos (centrada)
- Objetivo: Mitigar efectos de cuantización del contador de energía

In [ ]:
print("="*60)
print("CÁLCULO DE POTENCIA INSTANTÁNEA")
print("="*60)

# Calcular delta de energía (ya lo teníamos del Paso 4)
# df_tel['delta_energia'] ya existe

# Calcular delta de tiempo en horas
df_tel['delta_tiempo_h'] = df_tel.index.to_series().diff().dt.total_seconds() / 3600

# Calcular potencia en kW: P = ΔE / Δt
# Evitar división por cero
df_tel['P_kW'] = np.where(
    df_tel['delta_tiempo_h'] > 0,
    df_tel['delta_energia'] / df_tel['delta_tiempo_h'],
    0
)

# Suavizar potencia con media móvil de 5 segundos para mitigar cuantización
df_tel['P_kW_suavizada'] = df_tel['P_kW'].rolling(window=5, center=True, min_periods=1).mean()

# Mostrar resultados de las fórmulas aplicadas
print("\nPrimeros 10 valores calculados:")
print(df_tel[['energia_kwh', 'delta_energia', 'delta_tiempo_h', 'P_kW', 'P_kW_suavizada']].head(10))

print(f"\n✅ Potencia instantánea calculada")

PASO 2: Agregación a 1 minuto (telemetría)

### Fórmulas de agregación

Para cada minuto $m$:

$$\text{temp\_mean}(m) = \text{mean}(T)$$

$$\text{temp\_p95}(m) = \text{p95}(T)$$

$$\text{caudal\_mean}(m) = \text{mean}(q)$$

$$\text{P\_kW\_mean}(m) = \text{mean}(P)$$

$$\%\text{STOP}(m) = 100 \cdot \frac{\#\{i \in m : \text{estado}_i = \text{STOP}\}}{60}$$

Estas series minuto servirán como base para KPIs horarios/por turno.

In [ ]:
print("="*60)
print("AGREGACIÓN A 1 MINUTO (TELEMETRÍA)")
print("="*60)

# Crear agregaciones por minuto aplicando las fórmulas
df_1min = df_tel.resample('1min').agg({
    'temp_prod': ['mean', lambda x: x.quantile(0.95)],
    'caudal': 'mean',
    'P_kW': 'mean',
    'estado': lambda x: (x == 'STOP').sum()
}).round(3)

# Aplanar nombres de columnas
df_1min.columns = ['temp_mean', 'temp_p95', 'caudal_mean', 'P_kW_mean', 'segundos_stop']

# Calcular %STOP
df_1min['pct_STOP'] = ((df_1min['segundos_stop'] / 60) * 100).round(2)

print(f"\nPrimeros registros:")
print(df_1min.head(10))

print(f"\n✅ Agregación a 1 minuto completada")

PASO 3: Clasificación de botellas por tolerancia de peso

### Objetivo de masa por formato

$$m_{\text{obj}}(250) = 250\text{ g}, \quad m_{\text{obj}}(500) = 500\text{ g}$$

### Criterio de tolerancia

Con tolerancia típica del ±2%, una unidad está dentro de tolerancia si:

$$|\text{peso\_lleno\_g} - m_{\text{obj}}(f)| \leq 0.02 \cdot m_{\text{obj}}(f)$$

Donde:
- $\text{peso\_lleno\_g}$ es el peso neto de la botella (en gramos)
- $f$ es el formato (250 o 500)
- $m_{\text{obj}}(f)$ es la masa objetivo según el formato

In [ ]:
print("="*60)
print("CLASIFICACIÓN DE BOTELLAS POR TOLERANCIA")
print("="*60)

# Definir masa objetivo según formato
MASA_OBJ = {250: 250.0, 500: 500.0}
TOLERANCIA = 0.02  # ±2%

# Crear columna con masa objetivo según el formato de cada botella
df_pz['masa_objetivo'] = df_pz['formato_ml'].map(MASA_OBJ)

# Calcular desviación absoluta respecto al objetivo
df_pz['desviacion_abs'] = np.abs(df_pz['peso_lleno_g'] - df_pz['masa_objetivo'])

# Calcular límite de tolerancia (2% de la masa objetivo)
df_pz['limite_tolerancia'] = TOLERANCIA * df_pz['masa_objetivo']

# Clasificar: dentro_tolerancia = True si |peso_neto - m_obj| ≤ 0.02 * m_obj
df_pz['dentro_tolerancia'] = df_pz['desviacion_abs'] <= df_pz['limite_tolerancia']

# Mostrar resultados
print(f"\nPrimeros registros clasificados:")
print(df_pz[['id_botella', 'formato_ml', 'peso_lleno_g', 'masa_objetivo', 'desviacion_abs', 'dentro_tolerancia']].head(10))

print(f"\n✅ Clasificación completada")

## PASO 4: KPIs por hora y por turno

### Definiciones de KPIs

Sea $W$ la ventana temporal (hora o turno). Calculamos:

---

#### 1. **Throughput** (unidades/hora)

$$\text{Throughput}(W) = \frac{N_W}{\text{horas}(W)}$$

**Donde:**
- $N_W$ = número total de botellas producidas en $W$
- $\text{horas}(W)$ = 1 hora (ventanas horarias) o 8 horas (turnos)

---

#### 2. **Scrap** (% no conforme)

$$\text{Scrap}(W) = 100 \cdot \frac{NG_W}{OK_W + NG_W} \quad \text{(si } N_W > 0\text{; en otro caso NaN)}$$

**Donde:**
- $NG_W$ = botellas fuera de tolerancia
- $OK_W$ = botellas dentro de tolerancia

---

#### 3. **Tiempo en marcha** (horas)

$$\text{Tiempo en marcha}(W) = t_{\text{RUN}}(W)$$

**Calculado como:**
$$t_{\text{RUN}}(W) = \frac{\#\{i \in W : \text{estado}_i = \text{RUN}\}}{3600}$$

---

#### 4. **Energía específica** (Wh/unidad)

$$\text{Wh/ud}(W) = \frac{1000 \cdot \Delta E_{\text{kWh}}(W)}{N_W} \quad \text{(si } N_W > 0\text{; en otro caso NaN)}$$

**Donde:**
- $\Delta E_{\text{kWh}}(W)$ = energía consumida en la ventana $W$
- Factor 1000 para convertir kWh → Wh

---

#### 5. **% dentro de tolerancia**

$$\%\text{Tol}(W) = 100 \cdot \frac{\#\{\text{unidades en tolerancia}\}}{N_W} \quad \text{(si } N_W > 0\text{)}$$

---

### Agregación temporal

**Por hora:**
- Resample de telemetría: `resample('1h')`
- Resample de botellas: `set_index('ts_ciclo').resample('1h')`

**Por turno:**
- Definir función `asignar_turno(hora)`:
  - T1 (Mañana): 06:00 - 14:00
  - T2 (Tarde): 14:00 - 22:00
  - T3 (Noche): 22:00 - 06:00
- Agrupar con `groupby(['fecha', 'turno'])`

---

### Combinación de fuentes

Combinar DataFrames de **botellas** y **telemetría** usando:
````python
kpis = pd.concat([kpis_botellas, kpis_telemetria], axis=1)

In [ ]:
# PASO 11: Clasificación de botellas por tolerancia de peso
print("="*60)
print("CLASIFICACIÓN DE BOTELLAS POR TOLERANCIA")
print("="*60)

# ============================================================================
# PARTE A: KPIs POR HORA
# ============================================================================
print("\n--- KPIs POR HORA ---")

# 1. Preparar df_pz con índice temporal
df_pz_idx = df_pz.set_index('ts_ciclo')

# 2. Agregar botellas por hora
kpis_hora_pz = df_pz_idx.resample('1h').agg({
    'id_botella': 'count',  # N_W: Total de botellas
    'dentro_tolerancia': ['sum', lambda x: (~x).sum()]  # OK y NG
})

# Aplanar nombres de columnas
kpis_hora_pz.columns = ['N_botellas', 'OK', 'NG']

# 3. Agregar telemetría por hora
kpis_hora_tel = df_tel.resample('1h').agg({
    'estado': lambda x: (x == 'RUN').sum() / 3600,  # Tiempo en RUN (horas)
    'energia_kwh': ['first', 'last']  # Energía inicial y final
})

kpis_hora_tel.columns = ['horas_RUN', 'energia_ini', 'energia_fin']
kpis_hora_tel['delta_energia_kWh'] = kpis_hora_tel['energia_fin'] - kpis_hora_tel['energia_ini']

# 4. Combinar ambos DataFrames
kpis_hora = pd.concat([kpis_hora_pz, kpis_hora_tel], axis=1)

# 5. Calcular KPIs
kpis_hora['throughput_ud_h'] = np.where(
    kpis_hora['N_botellas'] > 0,
    kpis_hora['N_botellas'] / 1.0,  # Dividir por 1 hora
    np.nan
)

kpis_hora['scrap_pct'] = np.where(
    kpis_hora['N_botellas'] > 0,
    100 * kpis_hora['NG'] / kpis_hora['N_botellas'],
    np.nan
)

kpis_hora['energia_Wh_ud'] = np.where(
    kpis_hora['N_botellas'] > 0,
    1000 * kpis_hora['delta_energia_kWh'] / kpis_hora['N_botellas'],
    np.nan
)

kpis_hora['pct_tolerancia'] = np.where(
    kpis_hora['N_botellas'] > 0,
    100 * kpis_hora['OK'] / kpis_hora['N_botellas'],
    np.nan
)

# Redondear
kpis_hora = kpis_hora.round(2)

print(f"\nKPIs por hora calculados: {len(kpis_hora)} horas")
print(f"\nPrimeras horas:")
print(kpis_hora[['N_botellas', 'throughput_ud_h', 'scrap_pct', 'horas_RUN', 'energia_Wh_ud', 'pct_tolerancia']].head(10))

# ============================================================================
# PARTE B: KPIs POR TURNO
# ============================================================================
print("\n" + "="*60)
print("--- KPIs POR TURNO ---")

# Definir turnos (ejemplo: 06:00-14:00, 14:00-22:00, 22:00-06:00)
def asignar_turno(hora):
    if 6 <= hora < 14:
        return 'T1_Mañana'
    elif 14 <= hora < 22:
        return 'T2_Tarde'
    else:
        return 'T3_Noche'

# Asignar turno a cada botella
df_pz['turno'] = df_pz['ts_ciclo'].dt.hour.apply(asignar_turno)
df_pz['fecha'] = df_pz['ts_ciclo'].dt.date

# Asignar turno a telemetría
df_tel['turno'] = df_tel.index.hour.map(asignar_turno)
df_tel['fecha'] = df_tel.index.date

# Agregar por fecha y turno (botellas)
kpis_turno_pz = df_pz.groupby(['fecha', 'turno']).agg({
    'id_botella': 'count',
    'dentro_tolerancia': ['sum', lambda x: (~x).sum()]
})

kpis_turno_pz.columns = ['N_botellas', 'OK', 'NG']

# Agregar por fecha y turno (telemetría)
kpis_turno_tel = df_tel.groupby(['fecha', 'turno']).agg({
    'estado': lambda x: (x == 'RUN').sum() / 3600,
    'energia_kwh': ['first', 'last']
})

kpis_turno_tel.columns = ['horas_RUN', 'energia_ini', 'energia_fin']
kpis_turno_tel['delta_energia_kWh'] = kpis_turno_tel['energia_fin'] - kpis_turno_tel['energia_ini']

# Combinar
kpis_turno = pd.concat([kpis_turno_pz, kpis_turno_tel], axis=1)

# Calcular KPIs por turno (8 horas por turno)
kpis_turno['throughput_ud_h'] = np.where(
    kpis_turno['N_botellas'] > 0,
    kpis_turno['N_botellas'] / 8.0,
    np.nan
)

kpis_turno['scrap_pct'] = np.where(
    kpis_turno['N_botellas'] > 0,
    100 * kpis_turno['NG'] / kpis_turno['N_botellas'],
    np.nan
)

kpis_turno['energia_Wh_ud'] = np.where(
    kpis_turno['N_botellas'] > 0,
    1000 * kpis_turno['delta_energia_kWh'] / kpis_turno['N_botellas'],
    np.nan
)

kpis_turno['pct_tolerancia'] = np.where(
    kpis_turno['N_botellas'] > 0,
    100 * kpis_turno['OK'] / kpis_turno['N_botellas'],
    np.nan
)

# Redondear
kpis_turno = kpis_turno.round(2)

print(f"\nKPIs por turno calculados: {len(kpis_turno)} turnos")
print(f"\nPrimeros turnos:")
print(kpis_turno[['N_botellas', 'throughput_ud_h', 'scrap_pct', 'horas_RUN', 'energia_Wh_ud', 'pct_tolerancia']].head(10))

print(f"\n✅ KPIs por hora y turno calculados correctamente")

PASO 5: Cálculo del OEE - Opción A (Por tiempos y ciclo nominal)

### Fórmula del OEE

$$\text{OEE}(W) = \text{Availability}(W) \times \text{Performance}(W) \times \text{Quality}(W)$$

---

### Componentes del OEE

#### 1. Availability (Disponibilidad)

$$\text{Availability}(W) = \frac{t_{\text{RUN}}(W)}{t_{\text{plan}}(W)}$$

**Interpretación:** Proporción del tiempo planificado que la máquina estuvo en marcha.

---

#### 2. Performance (Rendimiento)

$$\text{Performance}(W) \approx \frac{t_{\text{nom}}(W)}{t_{\text{medio\_RUN}}(W)}$$

Donde:
- $t_{\text{nom}}(W)$ = tiempo de ciclo nominal ponderado por formato en $W$
- $t_{\text{medio\_RUN}}(W)$ = tiempo medio de ciclo durante RUN en $W$

**Interpretación:** Qué tan rápido producimos vs. la velocidad teórica.

---

#### 3. Quality (Calidad)

$$\text{Quality}(W) = \frac{OK_W}{OK_W + NG_W}$$

**Interpretación:** Proporción de piezas buenas sobre el total producido.

---

### Parámetros

- $t_{\text{nom}} = 1.5$ s/botella (equivale a 2400 botellas/hora)
- $t_{\text{plan}} = 1$ hora (para ventanas horarias) ó $8$ horas (para turnos)

---

### Consideraciones de implementación

- Si $N_W = 0$ o $t_{\text{RUN}}(W) = 0$ → devolver `NaN`
- Availability ya calculada en PASO 4 como `horas_RUN / horas_planificadas`
- Quality ya calculada en PASO 4 como `pct_tolerancia / 100`

In [ ]:
# PASO 12: Cálculo del OEE - Opción A
print("="*60)
print("CÁLCULO DEL OEE - OPCIÓN A")
print("="*60)

# ============================================================================
# PARÁMETROS
# ============================================================================
T_NOM = 1.5  # segundos/botella
HORAS_PLANIFICADAS_HORA = 1.0  # 1 hora
HORAS_PLANIFICADAS_TURNO = 8.0  # 8 horas por turno

print(f"\nParámetros:")
print(f"   t_nom: {T_NOM} s/botella")
print(f"   Horas planificadas (hora): {HORAS_PLANIFICADAS_HORA} h")
print(f"   Horas planificadas (turno): {HORAS_PLANIFICADAS_TURNO} h")

# ============================================================================
# PARTE A: OEE POR HORA
# ============================================================================
print("\n" + "="*60)
print("--- OEE POR HORA ---")

# 1. Availability = horas_RUN / horas_planificadas
kpis_hora['Availability'] = kpis_hora['horas_RUN'] / HORAS_PLANIFICADAS_HORA

# 2. Performance = t_nom / t_medio_RUN
#    t_medio_RUN = horas_RUN / N_botellas (en horas/botella)
#    Convertir t_nom a horas: 1.5 s = 1.5/3600 horas
kpis_hora['Performance'] = np.where(
    (kpis_hora['N_botellas'] > 0) & (kpis_hora['horas_RUN'] > 0),
    (T_NOM / 3600) / (kpis_hora['horas_RUN'] / kpis_hora['N_botellas']),
    np.nan
)

# 3. Quality = pct_tolerancia / 100
kpis_hora['Quality'] = kpis_hora['pct_tolerancia'] / 100

# 4. OEE = Availability × Performance × Quality
kpis_hora['OEE'] = kpis_hora['Availability'] * kpis_hora['Performance'] * kpis_hora['Quality']

# Convertir a porcentaje
kpis_hora['OEE_pct'] = (kpis_hora['OEE'] * 100).round(2)

print(f"\nPrimeras horas con OEE:")
print(kpis_hora[['N_botellas', 'Availability', 'Performance', 'Quality', 'OEE_pct']].head(10))

# ============================================================================
# PARTE B: OEE POR TURNO
# ============================================================================
print("\n" + "="*60)
print("--- OEE POR TURNO ---")

# 1. Availability = horas_RUN / horas_planificadas
kpis_turno['Availability'] = kpis_turno['horas_RUN'] / HORAS_PLANIFICADAS_TURNO

# 2. Performance = t_nom / t_medio_RUN
kpis_turno['Performance'] = np.where(
    (kpis_turno['N_botellas'] > 0) & (kpis_turno['horas_RUN'] > 0),
    (T_NOM / 3600) / (kpis_turno['horas_RUN'] / kpis_turno['N_botellas']),
    np.nan
)

# 3. Quality = pct_tolerancia / 100
kpis_turno['Quality'] = kpis_turno['pct_tolerancia'] / 100

# 4. OEE = Availability × Performance × Quality
kpis_turno['OEE'] = kpis_turno['Availability'] * kpis_turno['Performance'] * kpis_turno['Quality']

# Convertir a porcentaje
kpis_turno['OEE_pct'] = (kpis_turno['OEE'] * 100).round(2)

print(f"\nPrimeros turnos con OEE:")
print(kpis_turno[['N_botellas', 'Availability', 'Performance', 'Quality', 'OEE_pct']].head(10))

print(f"\n✅ OEE calculado (Opción A)")

# FASE 3 — Análisis numérico (NumPy puro)

## Preparación y notación

**Rejilla temporal:** Por-ciclo (cada botella alineada con telemetría más próxima)

**Variables continuas:**
- `T` = temp_prod (°C)
- `q` = caudal (ml/s)
- `P` = P_kW (kW)
- `tc` = tiempo_ciclo_s (s)

**Variable binaria:**
- `RUN ∈ {0,1}` (1 si en marcha)

**Error de llenado:**
- `e = peso_lleno_g - m_obj(f)` donde `m_obj(250)=250g`, `m_obj(500)=500g`

In [ ]:
# PASO 1.1: Calcular error de llenado
print("="*60)
print("FASE 3 - ANÁLISIS NUMÉRICO (NumPy puro)")
print("="*60)
print("\nPASO 1.1: Cálculo del error de llenado")

# e = peso_lleno_g - m_obj(f)
df_pz['error_llenado'] = df_pz['peso_lleno_g'] - df_pz['masa_objetivo']

print(f"\nError de llenado calculado:")
print(f"   Media: {df_pz['error_llenado'].mean():.3f} g")
print(f"   Std: {df_pz['error_llenado'].std():.3f} g")
print(f"   Min: {df_pz['error_llenado'].min():.3f} g")
print(f"   Max: {df_pz['error_llenado'].max():.3f} g")

print(f"\n✅ Error de llenado calculado")

# PASO 1.2: Calcular tiempo de ciclo
print("\n" + "="*60)
print("PASO 1.2: Cálculo del tiempo de ciclo")

# tc = diferencia temporal entre botellas consecutivas (en segundos)
df_pz['tiempo_ciclo_s'] = df_pz['ts_ciclo'].diff().dt.total_seconds()

# Estadísticas (ignorar primer valor NaN)
tc_validos = df_pz['tiempo_ciclo_s'].dropna()

print(f"\nTiempo de ciclo calculado:")
print(f"   Media: {tc_validos.mean():.3f} s")
print(f"   Mediana: {tc_validos.median():.3f} s")
print(f"   Std: {tc_validos.std():.3f} s")
print(f"   Min: {tc_validos.min():.3f} s")
print(f"   Max: {tc_validos.max():.3f} s")

print(f"\n✅ Tiempo de ciclo calculado")


# PASO 1.3: Alinear telemetría con botellas
print("\n" + "="*60)
print("PASO 1.3: Alineación telemetría con botellas (merge_asof)")

# Preparar df_tel: asegurar que el índice se llame 'ts' y reset_index
df_tel.index.name = 'ts'  # Asegurar nombre del índice
df_tel_temp = df_tel[['temp_prod', 'caudal', 'P_kW', 'estado']].reset_index()

# Alinear cada botella con la muestra de telemetría más cercana ANTES del ciclo
df_merge = pd.merge_asof(
    df_pz.sort_values('ts_ciclo'),
    df_tel_temp.sort_values('ts'),
    left_on='ts_ciclo',
    right_on='ts',
    direction='backward',
    tolerance=pd.Timedelta(seconds=5)  # Máximo 5 segundos de diferencia
)

# Convertir estado a binario: RUN=1, STOP=0
df_merge['RUN'] = (df_merge['estado'] == 'RUN').astype(int)

print(f"\nAlineación completada:")
print(f"   Total de botellas: {len(df_merge):,}")
print(f"   Botellas con telemetría válida: {df_merge['temp_prod'].notna().sum():,}")

print(f"\nPrimeros registros alineados:")
print(df_merge[['ts_ciclo', 'temp_prod', 'caudal', 'P_kW', 'tiempo_ciclo_s', 'error_llenado', 'RUN']].head(10))

print(f"\n✅ Datos alineados")


# PASO 1.4: Extraer arrays NumPy y crear máscara de datos válidos
print("\n" + "="*60)
print("PASO 1.4: Extracción a NumPy y máscara de validez")

# Extraer arrays NumPy (a partir de aquí solo NumPy)
T = df_merge['temp_prod'].values
q = df_merge['caudal'].values
P = df_merge['P_kW'].values
tc = df_merge['tiempo_ciclo_s'].values
e = df_merge['error_llenado'].values
RUN = df_merge['RUN'].values

# Crear máscara de datos válidos (sin NaN en ninguna variable)
mask_validos = ~(np.isnan(T) | np.isnan(q) | np.isnan(P) | np.isnan(tc) | np.isnan(e))

# Filtrar arrays con la máscara
T_clean = T[mask_validos]
q_clean = q[mask_validos]
P_clean = P[mask_validos]
tc_clean = tc[mask_validos]
e_clean = e[mask_validos]
RUN_clean = RUN[mask_validos]

n_total = len(T)
n_validos = len(T_clean)

print(f"\nMáscara de datos válidos:")
print(f"   Total de registros: {n_total:,}")
print(f"   Registros válidos (sin NaN): {n_validos:,} ({n_validos/n_total*100:.2f}%)")
print(f"   Registros con NaN: {n_total - n_validos:,}")

print(f"\n✅ Arrays NumPy preparados: T, q, P, tc, e, RUN")

## PASO 2: Correlaciones de Pearson

### Fórmula de correlación de Pearson

$$r_{xy} = \frac{\text{cov}(x,y)}{\sigma_x \sigma_y} = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum(x_i-\bar{x})^2} \sqrt{\sum(y_i-\bar{y})^2}}$$

**Implementación con estandarización:**

1. Estandarizar: $z = \frac{x - \bar{x}}{\sigma_x}$
2. Matriz de correlación: $\mathbf{R} = \frac{\mathbf{X}_{\text{std}}^T \mathbf{X}_{\text{std}}}{n-1}$

donde $\mathbf{X}_{\text{std}}$ es la matriz de datos estandarizados.

In [ ]:
# PASO 2: Correlaciones de Pearson (NumPy puro)
print("="*60)
print("PASO 2: CORRELACIONES DE PEARSON")
print("="*60)

# Función para estandarizar (z-score)
def estandarizar(x):
    """Estandariza un array: (x - media) / std"""
    return (x - np.mean(x)) / np.std(x)

# Estandarizar todas las variables
T_std = estandarizar(T_clean)
q_std = estandarizar(q_clean)
P_std = estandarizar(P_clean)
tc_std = estandarizar(tc_clean)
e_std = estandarizar(e_clean)

# Construir matriz de datos estandarizados [T, q, P, tc, e]
X_std = np.column_stack([T_std, q_std, P_std, tc_std, e_std])

# Calcular matriz de correlación: R = (X'X) / (n-1)
n = len(T_clean)
corr_matrix = (X_std.T @ X_std) / (n - 1)

# Nombres de variables
var_names = ['T', 'q', 'P', 'tc', 'e']

print(f"\nMatriz de correlación de Pearson ({n:,} muestras):")
print("\n" + " "*8 + "".join(f"{v:>8}" for v in var_names))
print("-" * 48)
for i, nombre in enumerate(var_names):
    fila = "".join(f"{corr_matrix[i,j]:>8.3f}" for j in range(len(var_names)))
    print(f"{nombre:>8}{fila}")

# Identificar correlaciones más fuertes con el error (e)
print(f"\nCorrelaciones con el error de llenado (e):")
idx_e = 4  # Índice de 'e' en var_names
for i, var in enumerate(var_names[:-1]):  # Excluir 'e' mismo
    print(f"   {var} vs e: {corr_matrix[i, idx_e]:>7.3f}")

print(f"\n✅ Correlaciones calculadas con NumPy puro")

## PASO 3: Regresión Lineal OLS (NumPy puro)

### Objetivo

Ajustar un **modelo de regresión lineal múltiple** para explicar el **error de llenado** `e` en función de las variables físicas del proceso.

---

### Modelo matemático

$$e = \beta_0 + \beta_1 T + \beta_2 q + \beta_3 P + \beta_4 \text{RUN} + \varepsilon$$

**Donde:**
- **Variable dependiente (y):** `e` = error de llenado (gramos)
- **Variables independientes (X):**
  - `T` = Temperatura del producto (°C)
  - `q` = Caudal (ml/s)
  - `P` = Potencia instantánea (kW)
  - `RUN` = Estado de la máquina (1=marcha, 0=parada)
- **ε:** Error aleatorio

---

### Fórmula de estimación OLS

**Matriz de diseño:**

$$\mathbf{X} = \begin{bmatrix} 
1 & T_1 & q_1 & P_1 & \text{RUN}_1 \\
1 & T_2 & q_2 & P_2 & \text{RUN}_2 \\
\vdots & \vdots & \vdots & \vdots & \vdots \\
1 & T_n & q_n & P_n & \text{RUN}_n
\end{bmatrix}, \quad
\mathbf{y} = \begin{bmatrix} e_1 \\ e_2 \\ \vdots \\ e_n \end{bmatrix}$$

**Estimador de mínimos cuadrados:**

$$\hat{\boldsymbol{\beta}} = (\mathbf{X}^T \mathbf{X})^{-1} \mathbf{X}^T \mathbf{y}$$

---

### Métricas de calidad del ajuste

**1. Coeficiente de determinación R²:**

$$R^2 = 1 - \frac{SS_{\text{res}}}{SS_{\text{tot}}}$$

Donde:
- $SS_{\text{res}} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2$ = Suma de cuadrados de residuos
- $SS_{\text{tot}} = \sum_{i=1}^{n} (y_i - \bar{y})^2$ = Suma de cuadrados total

**Interpretación:** % de varianza explicada por el modelo.

---

**2. R² ajustado:**

$$R^2_{\text{adj}} = 1 - (1 - R^2) \cdot \frac{n-1}{n-p-1}$$

Donde:
- $n$ = número de observaciones
- $p$ = número de predictores (sin contar intercepto)

**Interpretación:** Penaliza la adición de predictores que no mejoran significativamente el modelo.

---

### Diagnósticos mínimos

**Media de residuos:**

$$\bar{\varepsilon} = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i) \approx 0$$

**Interpretación física esperada de los coeficientes:**

| Coeficiente | Signo esperado | Razón física |
|-------------|----------------|--------------|
| β₁ (T) | **Negativo (−)** | ↑ temperatura → ↓ viscosidad → fluye más rápido → subllenado |
| β₂ (q) | **Positivo (+)** | ↑ caudal → mayor flujo → sobrellenado |
| β₃ (P) | **≈ 0 o pequeño** | Efecto indirecto; relacionado con velocidad de cinta |
| β₄ (RUN) | **Negativo (−)** | Transiciones STOP→RUN pueden capturar inestabilidades |

---

### Implementación

El código aplicará:
1. Construcción de matrices **X** (con columna de unos) e **y**
2. Resolución del sistema **X'Xβ = X'y** usando `np.linalg.solve()`
3. Cálculo de predicciones **ŷ = Xβ**
4. Cálculo de residuos **ε = y - ŷ**
5. Métricas **R²** y **R²_adj**
6. Interpretación de signos y magnitudes

In [ ]:

# PASO 3: Regresión Lineal OLS (NumPy puro) - CON DIAGNÓSTICO
print("="*60)
print("PASO 3: REGRESIÓN LINEAL OLS (NumPy puro)")
print("="*60)

# ============================================================================
# 3.1: CONSTRUCCIÓN DE MATRICES X e y
# ============================================================================
print("\n--- 3.1: Construcción de matrices ---")

# Matriz de diseño X: [1, T, q, P, RUN]
n = len(T_clean)
X_ols = np.column_stack([
    np.ones(n),      # β₀ (intercepto)
    T_clean,         # β₁ (temperatura)
    q_clean,         # β₂ (caudal)
    P_clean,         # β₃ (potencia)
    RUN_clean        # β₄ (estado RUN/STOP)
])

y_ols = e_clean

print(f"Matriz X: {X_ols.shape} (n={n}, p={X_ols.shape[1]-1} predictores + intercepto)")
print(f"Vector y: {y_ols.shape}")

# ============================================================================
# DIAGNÓSTICO: Verificar varianza y colinealidad
# ============================================================================
print("\n--- DIAGNÓSTICO ---")

var_names = ['Intercepto', 'T', 'q', 'P', 'RUN']
print(f"\nEstadísticas de cada columna de X:")
for i, name in enumerate(var_names):
    col = X_ols[:, i]
    print(f"   {name:12s}: min={np.min(col):>8.3f}, max={np.max(col):>8.3f}, "
          f"std={np.std(col):>8.3f}, unique={len(np.unique(col)):>5}")

# Verificar si hay columnas constantes (std ≈ 0)
stds = np.std(X_ols, axis=0)
columnas_constantes = np.where(stds < 1e-10)[0]

if len(columnas_constantes) > 1:  # > 1 porque intercepto siempre es constante
    print(f"\n⚠️  PROBLEMA: Columnas con varianza ≈0 detectadas:")
    for idx in columnas_constantes:
        print(f"   - {var_names[idx]} (std={stds[idx]:.10f})")
    print("\nSOLUCIÓN: Eliminar variable(s) constante(s)")

# Verificar condición de la matriz X'X
XtX = X_ols.T @ X_ols
cond_number = np.linalg.cond(XtX)
print(f"\nNúmero de condición de X'X: {cond_number:.2e}")

if cond_number > 1e10:
    print("⚠️  Matriz mal condicionada (multicolinealidad o columnas constantes)")
    print("   Solución: Usar pseudo-inversa (np.linalg.lstsq)")

# ============================================================================
# 3.2: ESTIMACIÓN DE COEFICIENTES β (con manejo robusto)
# ============================================================================
print("\n--- 3.2: Estimación de coeficientes ---")

try:
    # Intentar método estándar
    Xty = X_ols.T @ y_ols
    beta = np.linalg.solve(XtX, Xty)
    metodo = "solve() directo"
    
except np.linalg.LinAlgError:
    # Si falla, usar mínimos cuadrados con pseudo-inversa
    print("⚠️  solve() falló (matriz singular)")
    print("   Usando np.linalg.lstsq() (pseudo-inversa)")
    
    beta, residuals, rank, s = np.linalg.lstsq(X_ols, y_ols, rcond=None)
    metodo = f"lstsq() - rank={rank}/{X_ols.shape[1]}"
    
    if rank < X_ols.shape[1]:
        print(f"   ⚠️  Rango deficiente: {rank}/{X_ols.shape[1]}")
        print(f"   Algunas variables pueden ser redundantes")

# Nombres de los coeficientes
coef_names = ['β₀ (intercepto)', 'β₁ (T)', 'β₂ (q)', 'β₃ (P)', 'β₄ (RUN)']

print(f"\nMétodo usado: {metodo}")
print(f"\nCoeficientes estimados:")
for name, coef in zip(coef_names, beta):
    print(f"   {name:20s}: {coef:>10.6f}")

# ============================================================================
# 3.3: PREDICCIONES Y RESIDUOS
# ============================================================================
print("\n--- 3.3: Predicciones y residuos ---")

y_pred = X_ols @ beta
residuos = y_ols - y_pred

print(f"\nEstadísticas de residuos:")
print(f"   Media: {np.mean(residuos):.6f} (debe ser ≈0)")
print(f"   Std: {np.std(residuos):.3f}")
print(f"   Min: {np.min(residuos):.3f}")
print(f"   Max: {np.max(residuos):.3f}")

# ============================================================================
# 3.4: MÉTRICAS DE CALIDAD DEL AJUSTE
# ============================================================================
print("\n--- 3.4: Calidad del ajuste ---")

SS_res = np.sum(residuos**2)
SS_tot = np.sum((y_ols - np.mean(y_ols))**2)
R2 = 1 - (SS_res / SS_tot)

p = X_ols.shape[1] - 1
R2_adj = 1 - (1 - R2) * (n - 1) / (n - p - 1)

print(f"\nR² = {R2:.4f} ({R2*100:.2f}% de varianza explicada)")
print(f"R²_adj = {R2_adj:.4f}")

# ============================================================================
# 3.5: INTERPRETACIÓN DE COEFICIENTES
# ============================================================================
print("\n--- 3.5: Interpretación física ---")

print(f"\nSignos esperados según teoría:")
print(f"   β₁ (T) < 0: ↑ temperatura → ↓ viscosidad → ↓ peso (subllenado)")
print(f"   β₂ (q) > 0: ↑ caudal → ↑ peso (sobrellenado)")
print(f"   β₃ (P) ≈ 0: Potencia tiene efecto indirecto")
print(f"   β₄ (RUN) < 0: Estado STOP puede capturar cambios de régimen")

print(f"\nSignos obtenidos:")
for i, (name, coef) in enumerate(zip(coef_names[1:], beta[1:]), 1):
    signo = "+" if coef > 0 else "-"
    coincide = "✓" if (
        (i == 1 and coef < 0) or  # β₁(T) negativo
        (i == 2 and coef > 0) or  # β₂(q) positivo
        (i == 4 and coef < 0)     # β₄(RUN) negativo
    ) else "✗"
    print(f"   {name:15s}: {signo} ({coef:>10.6f}) {coincide}")

print(f"\n✅ Regresión OLS completada")

# FASE 4 — Visualización (Matplotlib)

## Objetivo

Comunicar de forma clara el comportamiento de la línea y sus efectos en los KPIs mediante figuras reproducibles, legibles y con unidades explícitas.

---

## Estándares de presentación

### Formatos de guardado

- **PNG**: 150-200 DPI (para documentos/presentaciones)
- **SVG**: Vectorial (para escalabilidad sin pérdida)
- **Ubicación**: Carpeta `fig/`

### Tamaños de figura

- **Series temporales**: 10 × 4 pulgadas
- **Barras/histogramas**: 8 × 4 pulgadas

### Elementos obligatorios

✅ **Etiquetas completas** (variable + unidad)  
✅ **Cuadrícula discreta** (`grid(alpha=0.3)`)  
✅ **Ejes bien acotados** (evitar autoscaling extremo)  
✅ **Formato de tiempo** con `DateFormatter` y `HourLocator/MinuteLocator`  
✅ **`tight_layout()`** para evitar solapamiento  
✅ **Leyenda única** por figura (fuera del área si es necesario)  

### Convenciones de diseño

- **Anotaciones sobrias**: Solo hitos clave (p95, media, cambios de formato)
- **Consistencia visual**: Misma tipografía/tamaño en todas las figuras
- **Unidades siempre visibles** en ejes y leyendas

---

## PASO 0: Configuración del entorno de visualización

In [ ]:
# Importar matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

# Configuración global de matplotlib
plt.rcParams['figure.dpi'] = 100  # DPI para visualización en pantalla
plt.rcParams['savefig.dpi'] = 200  # DPI para guardado
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['figure.titlesize'] = 13

# Crear directorio para figuras si no existe
from pathlib import Path
fig_dir = Path("fig")
fig_dir.mkdir(exist_ok=True)


## PASO 1: Serie temporal 12-24h con temperatura, caudal y eventos

### Objetivo

Visualizar el comportamiento conjunto de **temperatura** y **caudal** durante un período operativo, identificando paradas e inestabilidades del proceso.

---

### Estructura de la figura

**🔵 Eje izquierdo (Y1): Temperatura del producto**
- Línea continua con `temp_prod` (°C)
- Banda ±σ con desviación estándar móvil (ventana 10 min)
  - Cálculo: `rolling(window=600, center=True).std()`
  - Visualización: `fill_between()` con transparencia

**🟠 Eje derecho (Y2): Caudal**
- Línea continua con `caudal` (ml/s)
- Compartiendo eje X con temperatura

**🔴 Anotaciones de eventos:**
- **Intervalos STOP** (micro_parada, limpieza, cambio_formato):
  - Sombreado con `axvspan(color='red', alpha=0.15)`
- **Líneas verticales** en cambios de formato/limpieza:
  - `axvline(color='purple', linestyle='--')`
  - Etiqueta rotada 90° en el borde superior

---

### Formato del eje temporal

```python
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))
```

**Resultado esperado:**
- Eje X: `06:00`, `08:00`, `10:00`, ... (cada 2 horas)
- Marcas menores cada 30 minutos

---

### Leyenda combinada

Como usamos `twinx()`, debemos combinar las leyendas:

```python
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left')
```

---

### Elementos clave a verificar

✅ Banda ±σ visible y coherente con la señal  
✅ Zonas STOP claramente diferenciadas  
✅ Eje X legible (no solapamiento de etiquetas)  
✅ Unidades en ambos ejes Y  
✅ Guardado en PNG (200 DPI) y SVG  

In [ ]:
print("\n" + "="*60)
print("FIGURA 1: Serie temporal 12-24h (temperatura + caudal + eventos)")
print("="*60)

# Seleccionar ventana de 12 horas (ajustar según tus datos)
ts_inicio_fig1 = df_tel.index.min() + pd.Timedelta(hours=6)
ts_fin_fig1 = ts_inicio_fig1 + pd.Timedelta(hours=12)

# Filtrar datos
df_tel_fig1 = df_tel.loc[ts_inicio_fig1:ts_fin_fig1].copy()

# Calcular desviación estándar móvil (ventana 10 min = 600 segundos)
df_tel_fig1['temp_std'] = df_tel_fig1['temp_prod'].rolling(window=600, center=True).std()

# Crear figura
fig, ax1 = plt.subplots(figsize=(10, 4))

# ============================================================================
# EJE IZQUIERDO: Temperatura
# ============================================================================
color_temp = 'tab:blue'
ax1.set_xlabel('Tiempo')
ax1.set_ylabel('Temperatura (°C)', color=color_temp)

# Línea de temperatura
line1 = ax1.plot(df_tel_fig1.index, df_tel_fig1['temp_prod'], 
                 color=color_temp, linewidth=1.2, label='Temperatura', alpha=0.8)

# Banda ±σ
ax1.fill_between(df_tel_fig1.index,
                  df_tel_fig1['temp_prod'] - df_tel_fig1['temp_std'],
                  df_tel_fig1['temp_prod'] + df_tel_fig1['temp_std'],
                  color=color_temp, alpha=0.2, label='±1σ (ventana 10min)')

ax1.tick_params(axis='y', labelcolor=color_temp)
ax1.grid(True, alpha=0.3, linestyle='--')

# ============================================================================
# EJE DERECHO: Caudal
# ============================================================================
ax2 = ax1.twinx()
color_caudal = 'tab:orange'
ax2.set_ylabel('Caudal (ml/s)', color=color_caudal)

line2 = ax2.plot(df_tel_fig1.index, df_tel_fig1['caudal'],
                 color=color_caudal, linewidth=1.2, label='Caudal', alpha=0.8)

ax2.tick_params(axis='y', labelcolor=color_caudal)

# ============================================================================
# SOMBREAR INTERVALOS STOP
# ============================================================================
eventos_fig1 = df_evt[(df_evt['ts_ini'] >= ts_inicio_fig1) & 
                       (df_evt['ts_ini'] <= ts_fin_fig1)]

for idx, evento in eventos_fig1.iterrows():
    if evento['tipo'] in ['micro_parada', 'limpieza', 'cambio_formato']:
        ax1.axvspan(evento['ts_ini'], evento['ts_fin'], 
                    color='red', alpha=0.15, zorder=0)

# Líneas verticales en cambios de formato/limpieza
eventos_criticos = eventos_fig1[eventos_fig1['tipo'].isin(['cambio_formato', 'limpieza'])]
for idx, evento in eventos_criticos.iterrows():
    ax1.axvline(evento['ts_ini'], color='purple', linestyle='--', 
                linewidth=0.8, alpha=0.6)
    ax1.text(evento['ts_ini'], ax1.get_ylim()[1]*0.95, 
             evento['tipo'].replace('_', ' ').title(),
             rotation=90, verticalalignment='top', fontsize=8, color='purple')

# ============================================================================
# FORMATO DE TIEMPO EN EJE X
# ============================================================================
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax1.xaxis.set_major_locator(mdates.HourLocator(interval=2))
ax1.xaxis.set_minor_locator(mdates.MinuteLocator(interval=30))

# ============================================================================
# LEYENDA COMBINADA
# ============================================================================
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left', framealpha=0.9)

# ============================================================================
# TÍTULO Y AJUSTES FINALES
# ============================================================================
fig.suptitle('Evolución de temperatura y caudal (12h)', fontweight='bold')
fig.tight_layout()

# Guardar
fig.savefig(fig_dir / 'fig1_serie_temporal_12h.png', dpi=200, bbox_inches='tight')
fig.savefig(fig_dir / 'fig1_serie_temporal_12h.svg', format='svg', bbox_inches='tight')

plt.show()

print(f"\n✅ Figura 1 guardada en {fig_dir}/")

## PASO 2: Barras apiladas - OEE por turno (A/P/Q)

### Objetivo

Comparar los **tres componentes del OEE** entre turnos para identificar el factor limitante de cada período operativo.

---

### Fórmula del OEE (recordatorio)

$$\text{OEE} = \text{Availability} \times \text{Performance} \times \text{Quality}$$

**Donde cada componente está en el rango [0, 1]** (o 0-100%)

---

### Estructura de las barras apiladas

**Orden de apilamiento (de abajo hacia arriba):**

1. **🟢 Availability** (verde)
   - % del tiempo planificado que la máquina estuvo en RUN
   
2. **🔵 Performance** (azul)
   - % de rendimiento respecto al ciclo nominal
   
3. **🟠 Quality** (naranja)
   - % de unidades dentro de tolerancia

---

### Anotaciones obligatorias

**Dentro de cada segmento:**
- **Porcentaje del componente** centrado verticalmente
- Texto en **blanco** y **negrita**
- Tamaño de fuente: 9pt

**Encima de cada barra:**
- **OEE total (%)** en negro
- Posición: 2 unidades por encima del borde superior

**Debajo del eje X:**
- **N de botellas** producidas en el turno
- **Tiempo en RUN** (horas)
- Color gris, tamaño 8pt

---

### Configuración de ejes

```python
ax.set_ylim(0, 105)  # 0-100% + margen para etiquetas
ax.set_yticks(np.arange(0, 101, 10))  # Ticks cada 10%
```

---

### Ejemplo de interpretación

Si un turno tiene:
- A = 80%, P = 90%, Q = 95%
- OEE = 0.80 × 0.90 × 0.95 = **68.4%**

**Lectura:**
- La barra verde ocupa 80 unidades
- La azul arranca en 80 y termina en 80+90 = 170
- La naranja arranca en 170 y termina en 170+95 = 265

⚠️ **Nota:** Esto es incorrecto. En barras apiladas de OEE, debemos mostrar las **pérdidas acumuladas**, no los porcentajes directos.

**Alternativa correcta (pérdidas en cascada):**
- Availability: 80% (pérdida de 20% por paradas)
- Performance: 90% de 80% = 72% (pérdida adicional de 8%)
- Quality: 95% de 72% = 68.4% (pérdida adicional de 3.6%)

**Implementación:**
Representar cada componente como el % que "sobrevive" después de aplicar el anterior.

---

### Elementos a verificar

✅ Orden consistente (A abajo, Q arriba)  
✅ Etiquetas % legibles en cada segmento  
✅ OEE total visible encima  
✅ Información de contexto (N, t_RUN) debajo  
✅ Leyenda clara  

In [ ]:
print("\n" + "="*60)
print("FIGURA 2: Barras apiladas - OEE por turno")
print("="*60)

# Preparar datos: convertir componentes del OEE a porcentaje
kpis_turno_fig2 = kpis_turno.copy()
kpis_turno_fig2['A_pct'] = (kpis_turno_fig2['Availability'] * 100).round(1)
kpis_turno_fig2['P_pct'] = (kpis_turno_fig2['Performance'] * 100).round(1)
kpis_turno_fig2['Q_pct'] = (kpis_turno_fig2['Quality'] * 100).round(1)

# Agrupar por turno (promediar todos los días)
oee_por_turno = kpis_turno_fig2.groupby('turno').agg({
    'A_pct': 'mean',
    'P_pct': 'mean',
    'Q_pct': 'mean',
    'OEE_pct': 'mean',
    'N_botellas': 'sum',
    'horas_RUN': 'sum'
}).round(1)

# Ordenar turnos
orden_turnos = ['T1_Mañana', 'T2_Tarde', 'T3_Noche']
oee_por_turno = oee_por_turno.reindex(orden_turnos)

# ============================================================================
# CREAR FIGURA
# ============================================================================
fig, ax = plt.subplots(figsize=(8, 4))

turnos = oee_por_turno.index
x_pos = np.arange(len(turnos))
width = 0.6

# Datos para barras apiladas
availability = oee_por_turno['A_pct'].values
performance = oee_por_turno['P_pct'].values
quality = oee_por_turno['Q_pct'].values

# ============================================================================
# BARRAS APILADAS (orden: A → P → Q)
# ============================================================================
bar1 = ax.bar(x_pos, availability, width, label='Availability', 
              color='#4CAF50', alpha=0.8)
bar2 = ax.bar(x_pos, performance, width, bottom=availability, 
              label='Performance', color='#2196F3', alpha=0.8)
bar3 = ax.bar(x_pos, quality, width, 
              bottom=availability + performance,
              label='Quality', color='#FF9800', alpha=0.8)

# ============================================================================
# ANOTACIONES: % dentro de cada segmento
# ============================================================================
for i, (a, p, q) in enumerate(zip(availability, performance, quality)):
    # Availability
    ax.text(i, a/2, f'{a:.1f}%', ha='center', va='center', 
            fontweight='bold', fontsize=9, color='white')
    
    # Performance
    ax.text(i, a + p/2, f'{p:.1f}%', ha='center', va='center',
            fontweight='bold', fontsize=9, color='white')
    
    # Quality
    ax.text(i, a + p + q/2, f'{q:.1f}%', ha='center', va='center',
            fontweight='bold', fontsize=9, color='white')

# ============================================================================
# ANOTACIÓN: OEE total encima de cada barra
# ============================================================================
for i, oee in enumerate(oee_por_turno['OEE_pct'].values):
    ax.text(i, availability[i] + performance[i] + quality[i] + 2,
            f'OEE: {oee:.1f}%', ha='center', va='bottom',
            fontweight='bold', fontsize=10, color='black')

# ============================================================================
# ANOTACIÓN: N de unidades y tiempo RUN (debajo del eje)
# ============================================================================
for i, (n, t_run) in enumerate(zip(oee_por_turno['N_botellas'], 
                                     oee_por_turno['horas_RUN'])):
    ax.text(i, -8, f'N={int(n):,}\n{t_run:.1f}h RUN',
            ha='center', va='top', fontsize=8, color='gray')

# ============================================================================
# CONFIGURACIÓN DE EJES Y LEYENDA
# ============================================================================
ax.set_ylabel('Componentes del OEE (%)')
ax.set_xlabel('Turno')
ax.set_xticks(x_pos)
ax.set_xticklabels([t.replace('_', '\n') for t in turnos])
ax.set_ylim(0, 105)
ax.set_yticks(np.arange(0, 101, 10))
ax.legend(loc='upper right', framealpha=0.9)
ax.grid(axis='y', alpha=0.3, linestyle='--')

# ============================================================================
# TÍTULO Y GUARDADO
# ============================================================================
fig.suptitle('OEE por turno - Desglose A/P/Q', fontweight='bold')
fig.tight_layout()

fig.savefig(fig_dir / 'fig2_oee_por_turno.png', dpi=200, bbox_inches='tight')
fig.savefig(fig_dir / 'fig2_oee_por_turno.svg', format='svg', bbox_inches='tight')

plt.show()

print(f"\n✅ Figura 2 guardada en {fig_dir}/")

## PASO 3: Histograma del error de llenado por formato

### Objetivo

Comparar la **distribución del error de llenado** entre formatos (250ml y 500ml), identificando asimetrías (subllenado/sobrellenado) y la proporción dentro de tolerancia.

---

### Variable analizada

$$e = \text{peso\_lleno\_g} - m_{\text{obj}}(f)$$

**Donde:**
- $m_{\text{obj}}(250) = 250$ g
- $m_{\text{obj}}(500) = 500$ g

**Interpretación:**
- $e > 0$ → Sobrellenado
- $e < 0$ → Subllenado
- $|e| \leq 0.02 \cdot m_{\text{obj}}$ → Dentro de tolerancia

---

### Regla de Freedman-Diaconis para selección de bins

$$h = 2 \cdot \frac{\text{IQR}}{n^{1/3}}$$

$$\text{n\_bins} = \left\lceil \frac{\text{max} - \text{min}}{h} \right\rceil$$

**Ventaja sobre bins fijos:**
- Se adapta a la variabilidad de los datos (IQR)
- Escala apropiadamente con el tamaño de muestra ($n^{1/3}$)
- Evita sobre-suavizado (pocos bins) o ruido excesivo (muchos bins)

---

### Elementos de la figura

**Dos subgráficos (sharey=True):**
- Izquierda: Formato 250ml
- Derecha: Formato 500ml

**Anotaciones verticales:**

1. **🟢 Banda de tolerancia** (`axvspan`)
   - 250ml: ±5g (±2% de 250g)
   - 500ml: ±10g (±2% de 500g)
   - Color verde con `alpha=0.15`

2. **🔴 Línea de media** (línea continua)
   - Cálculo: `error_llenado.mean()`
   - Etiqueta en leyenda con valor

3. **🟠 Línea P95** (línea punteada)
   - Cálculo: `error_llenado.quantile(0.95)`
   - Identifica valores extremos

**Cuadro de texto (esquina superior derecha):**
```
n = 12,345
85.3% en tolerancia
```

---

### Interpretación de la asimetría

**Asimetría positiva (cola derecha):**
- Media > Mediana
- Mayoría de botellas con sobrellenado leve
- Pocos casos extremos de sobrellenado

**Asimetría negativa (cola izquierda):**
- Media < Mediana
- Tendencia al subllenado
- Riesgo de no conformidad por peso insuficiente

**Distribución simétrica:**
- Media ≈ Mediana ≈ 0
- Sistema bien calibrado
- Colas balanceadas

---

### Elementos a verificar

✅ Bins calculados automáticamente (no hardcodeados)  
✅ Banda de tolerancia visible  
✅ Media y P95 marcadas  
✅ Estadísticas (n, % tol) en cuadro de texto  
✅ Eje X en gramos  
✅ Ejes Y compartidos para comparación directa  

In [ ]:
print("\n" + "="*60)
print("FIGURA 3: Histograma del error de llenado por formato")
print("="*60)

# ============================================================================
# PREPARAR DATOS POR FORMATO
# ============================================================================
df_250 = df_pz[df_pz['formato_ml'] == 250].copy()
df_500 = df_pz[df_pz['formato_ml'] == 500].copy()

# ============================================================================
# FUNCIÓN: Calcular número de bins (Freedman-Diaconis)
# ============================================================================
def freedman_diaconis_bins(data):
    """Calcula número de bins óptimo según Freedman-Diaconis"""
    q25, q75 = np.percentile(data, [25, 75])
    iqr = q75 - q25
    n = len(data)
    
    h = 2 * iqr / (n ** (1/3))  # Ancho de bin óptimo
    
    if h == 0:  # Evitar división por cero
        return 30
    
    n_bins = int(np.ceil((data.max() - data.min()) / h))
    
    # Limitar entre 10 y 100 bins
    return max(10, min(n_bins, 100))

# Calcular bins
bins_250 = freedman_diaconis_bins(df_250['error_llenado'].values)
bins_500 = freedman_diaconis_bins(df_500['error_llenado'].values)

print(f"\nNúmero de bins calculado (Freedman-Diaconis):")
print(f"   250ml: {bins_250} bins")
print(f"   500ml: {bins_500} bins")

# ============================================================================
# CREAR FIGURA CON DOS SUBGRÁFICOS
# ============================================================================
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

# ============================================================================
# SUBGRÁFICO 1: Formato 250ml
# ============================================================================
ax1.hist(df_250['error_llenado'], bins=bins_250, 
         color='skyblue', edgecolor='black', alpha=0.7)

# Banda de tolerancia ±2% de 250g = ±5g
tol_250 = 0.02 * 250
ax1.axvspan(-tol_250, tol_250, color='green', alpha=0.15, 
            label=f'Tolerancia ±{tol_250:.1f}g')

# Media y P95
media_250 = df_250['error_llenado'].mean()
p95_250 = df_250['error_llenado'].quantile(0.95)

ax1.axvline(media_250, color='red', linestyle='-', linewidth=2, 
            label=f'Media: {media_250:.2f}g')
ax1.axvline(p95_250, color='orange', linestyle='--', linewidth=2,
            label=f'P95: {p95_250:.2f}g')

# Texto con estadísticas
pct_tol_250 = (df_250['dentro_tolerancia'].sum() / len(df_250) * 100)
ax1.text(0.95, 0.95, 
         f'n = {len(df_250):,}\n{pct_tol_250:.1f}% en tolerancia',
         transform=ax1.transAxes, fontsize=9,
         verticalalignment='top', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax1.set_xlabel('Error de llenado (g)')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Formato 250ml')
ax1.legend(loc='upper left', fontsize=8)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# ============================================================================
# SUBGRÁFICO 2: Formato 500ml
# ============================================================================
ax2.hist(df_500['error_llenado'], bins=bins_500,
         color='lightcoral', edgecolor='black', alpha=0.7)

# Banda de tolerancia ±2% de 500g = ±10g
tol_500 = 0.02 * 500
ax2.axvspan(-tol_500, tol_500, color='green', alpha=0.15,
            label=f'Tolerancia ±{tol_500:.1f}g')

# Media y P95
media_500 = df_500['error_llenado'].mean()
p95_500 = df_500['error_llenado'].quantile(0.95)

ax2.axvline(media_500, color='red', linestyle='-', linewidth=2,
            label=f'Media: {media_500:.2f}g')
ax2.axvline(p95_500, color='orange', linestyle='--', linewidth=2,
            label=f'P95: {p95_500:.2f}g')

# Texto con estadísticas
pct_tol_500 = (df_500['dentro_tolerancia'].sum() / len(df_500) * 100)
ax2.text(0.95, 0.95,
         f'n = {len(df_500):,}\n{pct_tol_500:.1f}% en tolerancia',
         transform=ax2.transAxes, fontsize=9,
         verticalalignment='top', horizontalalignment='right',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

ax2.set_xlabel('Error de llenado (g)')
ax2.set_title('Formato 500ml')
ax2.legend(loc='upper left', fontsize=8)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

# ============================================================================
# TÍTULO GENERAL Y GUARDADO
# ============================================================================
fig.suptitle('Distribución del error de llenado por formato', fontweight='bold')
fig.tight_layout()

fig.savefig(fig_dir / 'fig3_histograma_error_llenado.png', dpi=200, bbox_inches='tight')
fig.savefig(fig_dir / 'fig3_histograma_error_llenado.svg', format='svg', bbox_inches='tight')

plt.show()

print(f"\n✅ Figura 3 guardada en {fig_dir}/")
print(f"\nInterpretación:")
print(f"   - Asimetría positiva → tendencia al sobrellenado")
print(f"   - Asimetría negativa → tendencia al subllenado")

## PASO 4: Scatter "binned" - Temperatura vs % en tolerancia

### Objetivo

Analizar la **relación entre temperatura del producto y calidad del llenado**, controlando por el efecto del caudal mediante codificación de color.

---

### Hipótesis física

$$\text{temp\_prod} \uparrow \implies \text{viscosidad} \downarrow \implies \text{flujo más rápido} \implies \text{subllenado}$$

**Esperado:**
- A mayor temperatura → menor % en tolerancia (si el sistema no compensa)
- El efecto puede ser modulado por el caudal programado

---

### Metodología de binning

**1. Dividir `temp_prod` en bins de 0.5°C:**

```python
bins_temp = np.arange(df['temp_prod'].min(), 
                      df['temp_prod'].max() + 0.5, 
                      0.5)
df['bin_temp'] = pd.cut(df['temp_prod'], bins=bins_temp)
```

**2. Para cada bin, calcular:**
- **Centro del bin**: `(límite_inferior + límite_superior) / 2`
- **% en tolerancia**: `(dentro_tolerancia.sum() / n) * 100`
- **Caudal medio**: `caudal.mean()`
- **Conteo**: `n` (número de botellas en el bin)

**3. Filtrar bins con n insuficiente:**
- Umbral mínimo: **n ≥ 30** (para estabilidad estadística)

---

### Elementos de la visualización

**Scatter plot:**
- **Eje X**: Centro del bin de temperatura (°C)
- **Eje Y**: % en tolerancia
- **Color**: Caudal medio (ml/s) con `colorbar`
- **Tamaño** (opcional): Proporcional a `n` (conteo del bin)

**Intervalo de confianza binomial (Wilson):**

Para cada bin, calcular IC del 95% para la proporción:

$$p \pm 1.96 \sqrt{\frac{p(1-p)}{n}}$$

Representar con **barras de error verticales** (`errorbar`)

**Colorbar:**
- Etiqueta: "Caudal medio (ml/s)"
- Paleta sugerida: `viridis` o `coolwarm`

---

### Fórmula del intervalo de confianza de Wilson

Más robusto que la aproximación normal, especialmente para $p$ cercano a 0 o 1:

$$\text{IC}_{95\%} = \frac{p + \frac{z^2}{2n} \pm z\sqrt{\frac{p(1-p)}{n} + \frac{z^2}{4n^2}}}{1 + \frac{z^2}{n}}$$

Donde $z = 1.96$ para 95% de confianza.

**Implementación simplificada (aproximación normal):**

```python
from scipy.stats import binom

def wilson_ci(p, n, z=1.96):
    """Intervalo de confianza de Wilson para proporción"""
    denominator = 1 + z**2/n
    center = (p + z**2/(2*n)) / denominator
    margin = z * np.sqrt((p*(1-p)/n + z**2/(4*n**2))) / denominator
    return center - margin, center + margin
```

---

### Interpretación esperada

**Escenario ideal:**
- Pendiente negativa: ↑ temperatura → ↓ % tolerancia
- Puntos con caudal alto (color cálido) pueden compensar parcialmente
- IC estrechos en bins con n grande

**Caso preocupante:**
- Caída abrupta del % tolerancia en cierto rango de temperatura
- Sugiere punto de inflexión donde el proceso pierde control

**Caso estable:**
- % tolerancia constante en todo el rango
- Sistema robusto a variaciones térmicas

---

### Elementos a verificar

✅ Solo bins con n ≥ 30  
✅ Barras de error (IC) visibles  
✅ Colorbar con unidades (ml/s)  
✅ Tamaño de puntos proporcional a n (si se implementa)  
✅ Eje X en °C, eje Y en %  
✅ Título descriptivo  

## PASO 5: Wh/ud por hora con hitos operativos

### Objetivo

Monitorear la **eficiencia energética específica** (Wh por unidad producida) a lo largo del tiempo, identificando horas de consumo anómalo y su relación con eventos operativos.

---

### Fórmula de energía específica

$$\text{Wh/ud}(h) = \frac{1000 \cdot \Delta E_{\text{kWh}}(h)}{N(h)}$$

**Donde:**
- $\Delta E_{\text{kWh}}(h)$ = Energía consumida en la hora $h$ (kWh)
- $N(h)$ = Número de botellas producidas en la hora $h$
- Factor 1000 para convertir kWh → Wh

**Consideraciones:**
- Si $N(h) = 0$ → No representar la barra (o marcar explícitamente)
- Si $N(h)$ es muy bajo (<30) → Valor poco representativo (alta variabilidad)

---

### Elementos de la figura

**📊 Barras por hora:**
- Altura = Wh/ud
- Color diferenciado (ej: azul para normal, rojo para Wh/ud > p95)

**📏 Línea de referencia horizontal:**
- **P95 de Wh/ud** como umbral de ineficiencia
- Estilo: línea punteada roja
- Etiqueta: `P95 = {valor:.1f} Wh/ud`

**📍 Hitos operativos (líneas verticales):**
- **Cambios de formato**: Línea púrpura con anotación
- **Limpiezas**: Línea naranja con anotación
- Posición de texto: arriba del eje, rotación 90°

**📈 Eje secundario (opcional):**
- **Throughput** (ud/h) como línea
- Permite correlacionar Wh/ud con nivel de carga

---

### Cálculo del percentil 95

```python
# Calcular solo sobre horas con producción
wh_ud_valido = kpis_hora[kpis_hora['N_botellas'] > 0]['energia_Wh_ud']
p95_wh_ud = wh_ud_valido.quantile(0.95)
```

---

### Tratamiento de horas sin producción

**Opción A: Ocultar completamente**
```python
kpis_hora_prod = kpis_hora[kpis_hora['N_botellas'] > 0]
```

**Opción B: Mostrar con marcador especial**
```python
# Barra en gris claro con altura mínima
ax.bar(hora, 0.1, color='lightgray', hatch='//', label='Sin producción')
```

**Recomendación:** Opción A (más limpia)

---

### Formato del eje temporal

```python
ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m\n%H:%M'))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=4))
```

**Resultado:**
```
24/11     24/11     24/11
06:00     10:00     14:00
```

---

### Anotaciones de hitos

```python
# Filtrar eventos de interés
eventos_hitos = df_evt[df_evt['tipo'].isin(['cambio_formato', 'limpieza'])]

for idx, evento in eventos_hitos.iterrows():
    ts = evento['ts_ini']
    
    # Línea vertical
    ax.axvline(ts, color='purple' if evento['tipo']=='cambio_formato' else 'orange',
               linestyle='--', linewidth=1.5, alpha=0.7)
    
    # Texto rotado
    ax.text(ts, ax.get_ylim()[1]*0.95, 
            evento['tipo'].replace('_', ' ').title(),
            rotation=90, verticalalignment='top', fontsize=8)
```

---

### Interpretación esperada

**Wh/ud normal:**
- Valores estables alrededor de la media
- Variación <10% respecto a la mediana

**Picos de Wh/ud:**
- **Tras cambios de formato**: Normal (rampas de estabilización)
- **En horas de baja carga** (N pequeño): Consumo base no diluido
- **Aleatorios persistentes**: Posible deriva del proceso (ej: resistencias de calentamiento envejecidas)

**Tendencias temporales:**
- **Aumento progresivo**: Degradación de eficiencia (ej: fricción por desgaste)
- **Saltos súbitos**: Cambio de configuración o fallo

---

### Elementos a verificar

✅ Solo horas con N > 0 (o marcadas explícitamente)  
✅ Línea P95 visible y etiquetada  
✅ Hitos operativos anotados  
✅ Eje X con fechas legibles  
✅ Eje Y en Wh/ud  
✅ Colorbar/leyenda si se usa eje secundario  